This notebook provides an example of how to define and run a job on AzureML using Spark and external Spark libraries such as `spark-nlp`. This notebook is the _control plane_, meaning it creates a connection to the AzureML workspace, defines the job, and submits the job.

**This Jupyter notebook should be run from within a compute instance on AzureML, in a Python kernel, specifically `Python 3.10 - SDK v2 (Python 3.10.11)`**. 

## Create a client connection to the AzureML workspace

The following cell creates a connection object called `azureml_client` which has a connection to the AzureML workspace.

In [1]:
from azure.ai.ml import MLClient, spark, Input, Output
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import UserIdentityConfiguration

Use this authentication mechanism if you are running this notebook from your compute instance within Azure Machine Learning:

In [ ]:
## Use this authentication when running the control plane from the AzureML Compute Instance

azureml_client = MLClient.from_config(
    DefaultAzureCredential(),
)

However, you can also run this control plane notebook from your Laptop. You need to install the python libraries in the `requirements.txt` file.

In [4]:
## Use this when running the control plane from your laptop
ml_client = MLClient(
    credential=DefaultAzureCredential(),
    workspace_name="prof-azureml",
    subscription_id="21ff0fc0-dd2c-450d-93b7-96eeb3699b22",
    resource_group_name="prof-azureml"
)

## Create a container image to use 

In [5]:
from azure.ai.ml.entities import Environment
environment_object = Environment(
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu22.04",
    conda_file="sparknlp-environment.yml",
    name="sparknl-python-env"
)
ml_client.environments.create_or_update(environment_object)

Environment({'arm_type': 'environment_version', 'latest_version': None, 'image': 'mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu22.04', 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'sparknl-python-env', 'description': None, 'tags': {}, 'properties': {'azureml.labels': 'latest'}, 'print_as_yaml': False, 'id': '/subscriptions/21ff0fc0-dd2c-450d-93b7-96eeb3699b22/resourceGroups/prof-azureml/providers/Microsoft.MachineLearningServices/workspaces/prof-azureml/environments/sparknl-python-env/versions/1', 'Resource__source_path': '', 'base_path': '/Users/marck/class/dsan6000/working-repos/spark-on-azureml', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x127306270>, 'serialize': <msrest.serialization.Serializer object at 0x127324c50>, 'version': '1', 'conda_file': {'dependencies': ['python=3.10.3', {'pip': ['spark-nlp']}]}, 'build': None, 'inference_config': None, 'os_type': 'Linux', 'c

## Download the Spark-NLP jar to your working directory to be able to to add to the job cluster.

You only need to do this once. 

In [1]:
# Download the spark-nlp jar and save it locally. This needs to be done before submitting a job.
import requests
response = requests.get("https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/jars/spark-nlp-assembly-5.5.1.jar")
with open("spark-nlp-assembly-5.5.1.jar", "wb") as f:
    f.write(response.content)

## Define the Job

The following cell defines the job. It is an object of [Spark Class](https://learn.microsoft.com/en-us/python/api/azure-ai-ml/azure.ai.ml.entities.spark?view=azure-python) that contains the required information to run a job:

For more information about the parameters used in the job definition, [read the documentation](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-submit-spark-jobs?view=azureml-api-2&tabs=sdk#submit-a-standalone-spark-job).

In [ ]:
sparknlp_job_def = spark(
    display_name="sample-sparknlp-job-with-added-jar",
    code="./",
    entry={"file": "sample-spark-nlp-job.py"},
    driver_cores=1,
    driver_memory="7g",
    executor_instances=1,
    executor_cores=1,
    executor_memory="7g",
        resources={
        "instance_type": "Standard_E4S_V3",
        "runtime_version": "3.4",
    },
    jars="spark-nlp-assembly-5.5.1.jar",
    environment="sparknl-python-env@latest",
    identity=UserIdentityConfiguration()
)

## Submit the job

The following cell takes the job you defined above and submits it. If you are submitting multiple jobs, you may want to create separate job definition objects for clarity. You can submit more than one job, just remember that each job will spin up a Spark cluster.

In [ ]:
sparknlp_job = ml_client.jobs.create_or_update(sparknlp_job_def)

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Your file exceeds 100 MB. If you exper

## Get the Job Studio URL

Once you submit the job, you can navigate to it in the AzureML Studio and monitor it's progress. There are ways to do it through the SDK but for now just use the Studio. These are unattended jobs, which means you can shut down this notebook and the Compute Instance, but the job will go through it's lifecycle:

- Job is submitted
- Job is queued
- Job is run
- Job completes (assuming no errors)

**Each job's Studio URL will be different.**

In [18]:
print(sparknlp_job.studio_url)

https://ml.azure.com/runs/upbeat_receipt_ngxxln2kfr?wsid=/subscriptions/21ff0fc0-dd2c-450d-93b7-96eeb3699b22/resourcegroups/prof-azureml/workspaces/prof-azureml&tid=fd571193-38cb-437b-bb55-60f28d67b643
